This file contains functions that were developed to support the solution of the SAA for the TOP problem.

In [1]:
import scipy
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def print_solution(instance):
    info = ""
    route = ""
    next_visit = ""
    for i in instance.N:
        is_visited = 0
        next2visit = -1
        for j in instance.N:
            for k in instance.M:
                if (abs(instance.x[i,j,k].value) >= 0.1): 
                    is_visited = k
                    next2visit = j
        info      += "n" + str(i) + " -> " + str(next2visit) + "(" + str(is_visited) + ") "
    return(info)

In [3]:
def print_solutionXvehicle(instance):
    for k in instance.M:
        route = " Vehicle " + str(k) + ":"
        totaltime = 0
        totalreward = 0
        for (i,j) in instance.A:
            if (abs(instance.x[i,j,k].value) >= 0.1): 
                totaltime += instance.time[i,j]
                totalreward += instance.utility[j]
                route += "(" + str(i) + "," + str(j) + ":" + \
                str(instance.utility[j]) + ")"
        route += " - total time " + str(round(totaltime,2))
        route += " - total reward (det) " + str(round(totalreward, 2))
        print(route)
    return()

In [4]:
def compute_variance(instance):
    reward_scn=np.zeros(instance.nScn)
    for s in instance.S:
        reward = 0
        for k in instance.M:
            # print("scn " + str(s) + " veh" + str(k) + ": " + str(instance.rL[k,s].value))
            reward += instance.r[k,s].value
        reward_scn[s-1] = reward
        #print("scn " + str(s) + " reward: " + str(reward_scn[s-1]))
    num_bins = 20
    arr = plt.hist(reward_scn, bins=num_bins)
    for i in range(num_bins):
        if arr[0][i] > 0:
            plt.text(arr[1][i],arr[0][i],str(round(arr[0][i],0)))
    plt.show()
    # Compute the average (for information purposes)
    avg_reward = np.mean(reward_scn)
    # Compute the variance
    var_reward = round(np.var(reward_scn, ddof=1),2) # Divide by (n-1)
    print("Variance Large Scn Tree: " + str(var_reward) + " (avg " + str(avg_reward) + " sd " + str(round(sqrt(var_reward),1)) +")\n")
    return(var_reward)
#compute_variance(instanceLarge) 

In [5]:
def print4simulator(instance):
    f = open('C:/Users/Asus/Desktop/simulador_top/rutas/'+instanceName+'.txt','w')
    for k in instance.M:
        route = "0"
        ini = 0
        while (ini != (instance.n.value-1)):
            for (i,j) in instance.A:
                if (abs(instance.x[ini,j,k].value) >= 0.1): 
                    route += ";" + str(j) 
                    ini = j
        print(route)
        if k != instance.M.last():
            route += '\n'
        f.write(route)
    f.close()
    return()

#print4simulator(instance)

In [6]:
def build_test2Run_file(instanceName, cVar):
    with open('./tests/test2Run2.txt','w') as f:
        f.write('# inst	LongSim var \n')
        nsims = np.repeat([25, 100, 500, 1000, 2000, 5000, 10000, 25000, 50000], 10)
        info = ''
        for s in nsims:
            info += instanceName + '\t' + str(s) + '\t' + str(cVar) + '\n'
        f.write(info)
    
    return()

In [7]:
def run_simulator(instanceName,x_val = None, y_val = None):
    # Move to the simulator directory
    os.chdir('./simulador_top')
    
    build_test2Run_file(instanceName, cVar) 
    # Run the simulator
    !simuladorTOP.jar
    # Return to original path
    os.chdir('./..')

    # Import the data
    df_profit = pd.read_csv('C:/Users/Asus/Desktop/simulador_top/ResumeSols.txt', delimiter = "\t", index_col=False, skiprows=[0], header = None, decimal=",")
    df_profit.columns = ["sol_type", "instance", "StoProfit"]

    df_sim = pd.read_csv('./simulador_top/tests/test2Run2.txt', delimiter = "\t", index_col=False, skiprows=[0], header = None)
    df_sim.columns = ["instance", "size", "var"]
    df_full = pd.concat([df_sim, df_profit], axis=1)
    
    df = df_full[df_full['size']<= 10000]
    
    # Plot the scatter plot
    x = df["size"]
    y = df["StoProfit"]

    # Plot Extra points?
    x_sol = np.array([x_val])
    y_sol = np.array([y_val])
    
    plt.scatter(x,y) # ,color='blue'
    plt.scatter(x_sol, y_sol, color = 'orange')
    plt.title('Simulator objective value estimates')
    plt.xlabel('Sample size')
    plt.ylabel('Stochastic Objective Function Estimate')
    plt.savefig('C:/Users/Asus/Desktop/simulador_top/output/'+instanceName.replace('.','_')+'_simulator.png')
    plt.show()
    
    obj_estimate = round(np.mean(y),2)
    return(obj_estimate)
    
#run_simulator(instance,20000,80)

In [8]:
# Mimmics the stoch_times function, but without the need of a model
def sim_stoch_times(time, c_var = 0.05):
    if (time < 0.0001): # Departure/Arrival depot
        return (0)
    avg_t = time
    var_t = time*c_var
    mu = np.log(avg_t) - 0.5*np.log(1+var_t/(avg_t**2))
    sigma = np.sqrt(np.log(1+var_t/(avg_t**2)))
    
    return(round(np.random.lognormal(mu,sigma),1))

In [9]:
# Plot matrix
def plot_time(data,tmax):
    num_plots = data.shape[1]
    fig, axs = plt.subplots(num_plots, sharex=True, sharey=True)
    fig.suptitle('Distribution of the travelling times (' + str(tmax) + " max time)")
    
    for a in range(num_plots):
        axs[a].hist(data[:,a])
        axs[a].axvline(tmax, c='r')
    plt.savefig('C:/Users/Asus/Desktop/simulador_top/output/'+instanceName.replace('.','_')+'_time.png')
    plt.close(fig)
    return()

def plot_reward(data):
    num_plots = data.shape[1]
    fig, axs = plt.subplots(num_plots, sharex=True, sharey=True)
    fig.suptitle('Distribution of the rewards per vehicle')
    for a in range(num_plots):
        axs[a].hist(data[:,a], width = 2)
    #plt.show()
    plt.savefig('C:/Users/Asus/Desktop/simulador_top/output/'+instanceName.replace('.','_')+'_reward_veh.png')
    plt.close(fig)
    
    # Plot the total rewards
    tot_reward = data.sum(axis=1)
    avg_reward = round(np.mean(tot_reward),1)

    fig2, ax2 = plt.subplots()
    ax2.hist(tot_reward, zorder = 1)
    ax2.scatter(avg_reward,0,color = "orange", zorder = 2)
    ax2.set_title("Distribution of the solution reward")
    plt.savefig('C:/Users/Asus/Desktop/simulador_top/output/'+instanceName.replace(".","_")+'_reward.png')
    plt.close(fig2)
    
    #print('Average reward: ', avg_reward)
    return(avg_reward)



In [10]:
def simulate_sol(instanceName, nSimulations = 10, plot_time_dist = False):
    dataFile = 'C:/Users/Asus/Desktop/data/Chao/' + instanceName + '.txt'
    myfile = open(dataFile, "r")
    
    line      = myfile.readline().strip().split(';')
    num_nodes = int(line[1])
    
    line      = myfile.readline().strip().split(';')
    num_veh   = int(line[1])
    
    line      = myfile.readline().strip().split(';')
    tmax      = float(line[1])
    myfile.close()
    
    node = pd.read_csv(dataFile, sep=";", names=['x','y','reward'], index_col=False, skiprows=[0,1,2])
    
    df_reward = np.zeros((nSimulations,num_veh))
    df_time   = np.zeros((nSimulations,num_veh))
    route_reward = np.zeros(num_veh) # Route reward if completed in time
    df_time_scn = np.zeros(nSimulations)
    max_arcs = 6
    
    if plot_time_dist:
        fig, axs = plt.subplots(num_veh, max_arcs, figsize=(20,10))
        fig.subplots_adjust(hspace=0.4)
        #fig.suptitle('Distribution of the travelling times per node (' + str(tmax) + " max time)")
        
    solutionFile = './simulador_top/rutas/'+instanceName+'.txt'
    with open(solutionFile) as f:
        lines = f.readlines() # list containing the nodes visited
        veh = 0

        for line in lines:
            line = line.strip().split(';')
            line = list(map(int, line))
            #print(line)
            arc = 0
            for n in line:
                x_des = node.at[n,'x']
                y_des = node.at[n,'y']
                n_des = n
                route_reward[veh] += node.at[n,'reward']
                
                if n!=0: # Depot is always node 0
                    time = round(math.sqrt( ((x_des-x_ori)**2)+((y_des-y_ori)**2) ),2)
                    for s in range(nSimulations):
                        df_time_scn[s] = sim_stoch_times(time, cVar)
                        df_time[s,veh] += df_time_scn[s]
                    if plot_time_dist & (arc < max_arcs):
                        print('Arc',n_ori,'-',n_des,'\t time: ',time,
                              '  =?= avg:',round(np.mean(df_time_scn),2),
                              ' c= ', cVar,
                              '\t var=c*time: ', round(cVar*time,2),
                              ' =?=',    round(np.var(df_time_scn),2))
                        axs[veh, arc].hist(df_time_scn, zorder = 1)
                        axs[veh, arc].scatter(np.mean(df_time_scn),0,color='orange', zorder = 2)
                        axs[veh, arc].scatter(time,1,color='red', zorder = 3)
                        axs[veh, arc].set_title(str(n_ori)+'-'+ str(n_des)+' t:'+str(time))
                    arc += 1
                x_ori = x_des
                y_ori = y_des
                n_ori = n_des
            # Compute statistics for this route
            for s in range(nSimulations):
                if df_time[s,veh] <= tmax:
                    df_reward[s,veh] = route_reward[veh]
            veh += 1
    if plot_time_dist:
        plt.savefig('../output/'+instanceName.replace('.','_')+'_arc_time.png')
        plt.show()
        
            
    # Compute statistics 
    tot_reward = df_reward.sum(axis=1)
    avg_reward = round(np.mean(tot_reward),1)
    std_reward = round(np.std(tot_reward),1)
    min_reward = min(tot_reward)
    max_reward = max(tot_reward)
    # Routes reliability
    route_rel = df_time <= tmax
    avg_reliability = round(100*np.mean(route_rel),1)

    if nSimulations >= 10000:
        plot_time(df_time,tmax)
        plot_reward(df_reward)
    
    return([avg_reward, std_reward, min_reward, max_reward, avg_reliability])

#[a,b,c,d,e] = simulate_sol(instanceName,10000,plot_time_dist = True)

In [11]:
def print_solver_info(instance, results):
    det_obj = round(instance.Total_Utility_Stoch(),2)
    gap = round(100*(results.problem.upper_bound- results.problem.lower_bound)/(results.problem.lower_bound),2)
    solver_time = round(results.solver.time,2)
    solver_status = str(results.solver.status)
    solver_termination = str(results.solver.termination_condition) 
    
    print(instanceName, "; ", len(instance.N), "; ", det_obj, "; ", gap, "; ", solver_time, "; ",
          solver_status, "; ",solver_termination,";" )
    return()

In [12]:
def plot_reward_instance(instance):
    reward=np.zeros((instance.m.value,instance.nScn))
    for k in instance.M:
        for s in instance.S:
            reward[k-1,s-1] = instance.r[k,s].value
    
    fig, axs = plt.subplots(4, sharex=True, sharey=True)
    fig.suptitle('Rewards per vehicle in each scenario')

    axs[0].hist(reward[0,], width = 2)
    axs[1].hist(reward[1,], width = 2)
    axs[2].hist(reward[2,], width = 2)
    axs[3].hist(reward[3,], width = 2)

    return(reward)

In [13]:
def plot_total_time(instance):
    route_time=np.zeros((instance.m.value,instance.nScn))
    route_max_time = instance.t_max.value
    for k in instance.M:
        nFail = 0
        for s in instance.S:
            #for i,j in instance.A:
            #    print(s,' time(',i,',',j,') = ',instance.time_stoch[i,j,s].value)
            route_time[k-1,s-1] = sum(instance.time_stoch[i,j,s].value*instance.x[i,j,k].value for (i,j) in instance.A) 
            if (route_time[k-1,s-1] > route_max_time):
                nFail = nFail + 1
        print("Vehicle " + str(k) + " Routes fulfilled " + 
              str(round(100*(instance.nScn-nFail)/instance.nScn,2)) + "% - " +
              str(nFail) + " failures out of " + str(instance.nScn))
    
    fig, axs = plt.subplots(4, sharex=True, sharey=True)
    fig.suptitle('Total time per route with stochastic travelling times (' + str(instance.t_max.value) + " max time)")

    axs[0].hist(route_time[0,])
    axs[0].axvline(instance.t_max.value, c='r')
    axs[1].hist(route_time[1,])
    axs[1].axvline(instance.t_max.value, c='r')
    axs[2].hist(route_time[2,])
    axs[2].axvline(instance.t_max.value, c='r')
    axs[3].hist(route_time[3,])
    axs[3].axvline(instance.t_max.value, c='r')

    return(route_time)

In [14]:
def save_tree(instance):
    outputFile = "../results/"+instanceName + "_s" + str(S) + "_c" + str(round(100*c)) + "_SAA_scn.csv"
    f = open(outputFile, "w")
    for i,j,s in instance.N*instance.N*instance.S:
        f.write(str(i) + ";" + str(j) + ";" + str(s) + ";" + str(instance.time_stoch[i,j,s].value) + "\n")
    f.close()
    return()

#save_tree(instanceLarge)